In [85]:
import pandas as pd
import unicodedata

In [86]:
df = pd.read_csv('whole_df_datasport.csv')

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
def toUpper(value):
    if not isinstance(value, str):
        return value
    if value.isupper():
        return value
    else:
        nfkd_form = unicodedata.normalize('NFKD', value)
        return u"".join([c.upper() for c in nfkd_form if not unicodedata.combining(c)])

In [88]:
df['Nom'] = df['Nom'].apply(toUpper)

In [89]:
df['Categorie'] = df['Categorie'].apply(toUpper)

In [90]:
df['Lieu'] = df['Lieu'].apply(toUpper)

In [91]:
def to_seconds(time_str):
    if time_str.find('-') >= 0:
        return nan
    debug = False
    
    hours = 0
    mins = 0
    try:
        if time_str.find(':') > 0: # if more than an hour
            h_mm = time_str.split(':')
            hours = int(h_mm[0])
            time_str = h_mm[1]
        
        if time_str.find('.') < 1:
            print('Warning: no dot found in', time_str)
            return
        mm, ss = time_str.split('.')
        mins = int(mm)
        seconds = float(ss.replace(',', '.'))
        
        if debug:
            print(str(hours)+':'+str(mins)+'.'+str(seconds))
        
        return 3600*hours + 60*mins + seconds
    except ValueError as e:
        print('Error with', time_str, ':', e)
        return

In [92]:
def get_main_category(category):
    lastPartLen = len(category.split(' ')[-1])
    if lastPartLen < 4:
        return category[:-(lastPartLen+1)].title()
    else:
        return category.title()
def capitalize_cat(category):
    return category.title()
df['categorie_principale'] = df['Categorie'].apply(get_main_category)

In [93]:
df.head(2)

,Categorie,Rang,Nom,Naissance,Lieu,Groupe,Temps,Dossard,year,categorie_principale
0,POUSSINS B,1,WOLFENSBERGER JOEL,2001,COMMUGNY,ECOLE PRIMAIRE DE COPPET,"7.44,6",4301.0,2008,Poussins
1,POUSSINS B,2,TRIPONEZ LOIC,2001,BASSINS,STADE GENEVE ATHLETISME,"7.44,7",26021.0,2008,Poussins


In [94]:
df['temps_secondes'] = df['Temps'].apply(to_seconds)

In [96]:
df.columns = [i.lower() for i in df.columns]

In [ ]:
#for year, year_df in df_katno.groupby('year'):
#    for i, group in year_df.groupby('catégorie'):
        #if len(set(group['katno'].values)) == 1:
        #    katnos[i] =  group['katno'].values[0]
        #else:
        #    print("error with", group['katno'].values)
        #    raise('Fatal.')
# -> foireux: change d'une année à l'autre... et meme dans une meme annee

In [97]:
katnos_list = []
# on fait notre propre systeme
c = 1000
for i, group in df_katno.groupby('catégorie'):
    katnos_list.append(i)
katnos_list = [toUpper(i) for i in katnos_list]



In [98]:
katnos = {}
c = 1000
for item in katnos_list:
    katnos[item] = c
    c += 1

In [99]:
def add_katno(categorie):
    return katnos[categorie]

In [100]:
df['katno'] = df['categorie'].apply(add_katno)

In [101]:
df.columns

Index(['categorie', 'rang', 'nom', 'naissance', 'lieu', 'groupe', 'temps',
       'dossard', 'year', 'categorie_principale', 'temps_secondes', 'katno'],
      dtype='object')

In [102]:
df = df[['katno', 'categorie', 'categorie_principale', 'rang', 'nom', 'naissance', 'temps', 'year', 'temps_secondes']]

In [104]:
df.columns = ['katno', 'categorie', 'categorie_principale', 'rang', 'nom',
       'naissance', 'temps_str', 'year', 'temps_secondes']

In [107]:
lastRankIndexes = []

def is_float(value):
    if value != value:
        return False
    if isinstance(value, float):
        return True
    elif isinstance(value, int):
        return True
    else:
        return False
for h, x in df[df['temps_secondes'].apply(is_float) == True].groupby('year'):
    for i, y in x.groupby('categorie'):
    #for i, y in df[df['temps_secondes'].apply(is_float) == True].groupby('catégorie'):
        index = y['rang'].idxmax()
        lastRankIndexes.append(index)
        print(y.ix[index][['rang', 'temps_str']])
print(len(lastRankIndexes))

rang             256
temps_str    28.51,8
Name: 4494, dtype: object
rang             421
temps_str    37.24,4
Name: 4238, dtype: object
rang             201
temps_str    19.40,1
Name: 14829, dtype: object
rang             355
temps_str    19.53,0
Name: 14628, dtype: object
rang             563
temps_str    19.58,3
Name: 14273, dtype: object
rang             828
temps_str    20.18,0
Name: 13710, dtype: object
rang             670
temps_str    22.57,3
Name: 3817, dtype: object
rang             996
temps_str    20.16,6
Name: 3147, dtype: object
rang             205
temps_str    33.05,9
Name: 15148, dtype: object
rang             525
temps_str    46.21,5
Name: 5177, dtype: object
rang             727
temps_str    39.36,3
Name: 15875, dtype: object
rang             938
temps_str    43.33,7
Name: 16813, dtype: object
rang            1021
temps_str    40.40,1
Name: 17834, dtype: object
rang             356
temps_str    39.30,0
Name: 18190, dtype: object
rang              97
temps_str    40.33

In [108]:
df['is_last'] = 0

In [109]:
for i in lastRankIndexes:
    # df.set_value(index, col, value, takeable=False)
    df.set_value(i, 'is_last', 1)

In [110]:
df.to_csv('ready_for_sql.csv', index=False)